# Averaging, summarizing, and displaying connectivity models
Connectivity models are estimated for each participant individually - The target structure ($\mathbf{Y}$, cerebellum) is predicted on a voxel/vertex level from the source structure ($\mathbf{X}$, neocortex), which is parcellated at a certain granularity. The files are store as .h5 files in folder named after the target atlas. 



In [ ]:
# Import necessary modules
import numpy as np
import deepdish as dd
import nibabel as nb
import cortico_cereb_connectivity.globals as gl
import cortico_cereb_connectivity.run_model as rm
import cortico_cereb_connectivity.cio as cio
import json

###  Averaging across participants

In [ ]:
# Average a specific model across participants
train_data = 'MDTB'                 # Training data 
train_ses= "ses-s1"                 # Training session 
parcellation = 'Icosahedron162'     # Source structure parcellation
method='NNLS'                       # Method
parameters=['coef_']                # Parameters to averae
cerebellum='SUIT3'                  # Target structure (cerebellum) atlas
ext = 'A4'                          # Extension 
avg_id = 'avg'                      # ID for the average model

mname_base = f"{train_data}_{train_ses}_{parcellation}_{method}"
model_path = gl.conn_dir + f"/{cerebellum}/train/{mname_base}/"
avrg_model,info = rm.calc_avrg_model(train_data,
            mname_base,
            ext,
            cerebellum=cerebellum,
            parameters=parameters,
            avrg_mode='avr_sep')
fname = model_path + f"/{mname_base}{ext}_{avg_id}" # Model basename 
cio.save_model(avrg_model,info,fname)

### Exporting model to pdconn.nii file
``export_model`` is currently a script in ``scripts/script_export_model.py``. For specific exports, it is best to write your own scipt to call ``Model.to_cifti``. 

In [ ]:
def export_model(model_name,model_ext,file_name,type='pdconn'):
    model_path = os.path.join(gl.conn_dir,'SUIT3','train',model_name)
    fname = model_path + f"/{model_name}_{model_ext}"
    M,info = cio.load_model(fname)
    adir = am.default_atlas_dir
    src_roi = [f"{adir}/tpl-{info['cortex']}/Icosahedron1002.L.label.gii",
               f"{adir}/tpl-{info['cortex']}/Icosahedron1002.R.label.gii"]
    C = M.to_cifti(src_atlas=info['cortex'],
                    trg_atlas=info['cerebellum'],
                    src_roi=src_roi,
                    fname=f'data/{file_name}',
                    dtype = 'float32')

In [ ]:
export_model('MDTB_all_Icosahedron1002_L2regression','A8_avg','Nettekoven_2024_MDTB_L2.pdconn.nii')

## Averaging connectivity models by target-structure ROI
For visualization it is useful to average the connectivity weights also by a specific target-structure ROI. 
This script will produce a .pscalar file 

In [ ]:
from cortico_cereb_connectivity.scripts.script_summarize_weights import avrg_weight_map,sort_roi_rows

cifti_img = avrg_weight_map(method = 'NNLS',
                                cortex_roi = "Icosahedron162",
                                cerebellum_roi = "NettekovenSym32",
                                cerebellum_atlas = "SUIT3",
                                extension = 'A4',
                                dataset_name = 'MDTB',
                                ses_id = "ses-s1",
                                )
fname = gl.conn_dir + f'/maps/MDTBs1_NNLS162_A4.pscalar.nii'
cifti_img = sort_roi_rows(cifti_img)
nb.save(cifti_img,fname)